### Interactive Kymograph Code

Use this notebook to figure out the kymograph hyperparameters that are most appropriate for your dataset.

In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider, IntSlider
import ipywidgets as widgets
import matplotlib
import warnings
matplotlib.rcParams['figure.figsize'] = [20, 10]
warnings.filterwarnings(action='once')

In [2]:
import trenchripper as tr

/home/de64/anaconda3/envs/mothermachine/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/de64/anaconda3/envs/mothermachine/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/de64/anaconda3/envs/mothermachine/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/de64/anaconda3/envs/mothermachine/lib/python3.6/site-packages/skimage/__init__.py:71: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/de64/anaconda3/envs/mothermachine/lib/python3.6/site-packages/pytest.py' mode='r' encoding='utf-8'>
  imp.find_module('pytest')
/home/de64/anaconda3/envs/mothermachine/lib/python3.6/importlib/_bootstrap.py:219:

In [3]:
import h5py
import numpy as np
import time

In [2]:
time.time()

1548035101.2690163

In [3]:
for i in range(7,11):
    print(2**i)
    with h5py.File('/n/scratch2/de64/test_'+str(i)+'.hdf5', 'a') as outfile:
        outfile.create_dataset("data", (2048,2048,300),chunks=(2**i, 2**i, 1), dtype='uint16')
        for j in range(3):
            array = np.ones((2048,2048,100),dtype='uint16')
            outfile["data"][:,:,(100*j):(100*j)+100] = array
    infile = h5py.File('/n/scratch2/de64/test_'+str(i)+'.hdf5', 'r')
    dataset = infile['data']
    t1 = time.time()
    testarr = dataset[:,:,0:300:100]
    t2 = time.time()
    print(t2-t1)

128
4.095885515213013
256
3.9577085971832275
512
3.777090311050415
1024
75.13803219795227


In [4]:
for i in range(0,4):
    print(2**i)
    with h5py.File('/n/scratch2/de64/test_'+str(i)+'.hdf5', 'a') as outfile:
        outfile.create_dataset("data", (2048,2048,300),chunks=(256, 256, 2**i), dtype='uint16')
        for j in range(3):
            array = np.ones((2048,2048,100),dtype='uint16')
            outfile["data"][:,:,(100*j):(100*j)+100] = array
    infile = h5py.File('/n/scratch2/de64/test_'+str(i)+'.hdf5', 'r')
    dataset = infile['data']
    t1 = time.time()
    testarr = dataset[:,:,0:300:100]
    t2 = time.time()
    print(t2-t1)

1
3.9162299633026123
2
3.9529340267181396
4
4.003893852233887
8
4.017016887664795


In [10]:
infile = h5py.File('/n/scratch2/de64/test_7.hdf5', 'r')

OSError: Unable to open file (unable to open file: name = '/n/scratch2/de64/test_7.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [4]:
2048/4

512.0

In [5]:
dataset = infile['data']

In [6]:
dataset.chunks

(256, 256, 1)

In [7]:
dataset.shape

(2048, 2048, 1400)

In [11]:
testarr = dataset[:,:,0:700:100]

In [9]:
testarr.nbytes

8388608

#### Initialize the interactive kymograph class

As a first step, initialize the `tr.interactive.kymograph_interactive` class that will be handling all steps of generating a kymograph. 

You will need to specify the following `args` and `kwargs` (in order):


**Args**

**input_file_prefix (string)** : File prefix for all input hdf5 files of the form "\[input_file_prefix\]\[number\].hdf5" This should be the default output format for the hdf5 export code, but you will need to rename files if taking input files from a different source.

**all_channels (list)** : list of strings corresponding to the different image channels available in the input hdf5 file, with the channel used for segmenting trenches in the first position. NOTE: these names must match those of the input hdf5 file datasets.

**fov_list (list)** : List of ints corresponding to the fovs that you wish to make kymographs of.

**Kwargs**

**t_subsample_step (int)** : Step size to be used for subsampling input files in time, recommend that subsampling results in between 5 and 20 timepoints for quick processing.

The last line will perform import and subsampling of the input hdf5 image files.

In [18]:
# input_path_prefix = '/n/groups/paulsson/Daniel/Image_analysis_pipeline/tiff_extraction/test_out_4/fov_'
# kymo = tr.interactive.kymograph_interactive(input_path_prefix,["channel_BF","channel_RFP"],[64,12,32,45],t_subsample_step=5)
# imported_array_list = kymo.map_to_fovs(kymo.import_hdf5)
input_path_prefix = '/n/scratch2/de64/full_pipeline_test/hdf5/fov_'
kymo = tr.interactive.kymograph_interactive(input_path_prefix,["channel_BF"],[1,2,3,4],t_subsample_step=200)
imported_array_list = kymo.map_to_fovs(kymo.import_hdf5)

#### Tune "trench-row" detection hyperparameters

The kymograph code begins by detecting the positions of trench rows in the image as follows:

1. Reducing each 2D image to a 1D signal along the y-axis by computing the qth percentile of the data along the x-axis
2. Smooth this signal using a median kernel
3. Use a [triangle threshold](https://imagej.net/Auto_Threshold#Triangle) to determine the trench row poisitons

This method uses the following `kwargs`, which you can tune here:

**y_percentile (int)** : Percentile to use for step 1.

**smoothing_kernel_y_dim_0 (int)** : Median kernel size to use for step 2.

**triangle_nbins (int)** : Number of bins to use in the triangle method histogram.

**triangle_scaling (float)** : Scaling factor to apply to the threshold determined by the triangle method.


Running the following widget will display the smoothed 1-D signal for each of your timepoints. In addition, the threshold value for each fov will be displayed as a red line.

In [19]:
matplotlib.rcParams['figure.figsize'] = [20, 10]
interact_manual(kymo.preview_y_precentiles, imported_array_list=fixed(imported_array_list),y_percentile=IntSlider(value=85, min=0, max=100, step=1),\
         smoothing_kernel_y_dim_0=IntSlider(value=9, min=1, max=20, step=1),triangle_nbins=IntSlider(value=50, min=10, max=200, step=10),\
                triangle_scaling=FloatSlider(value=1., min=0., max=2., step=0.05));

interactive(children=(IntSlider(value=85, description='y_percentile'), IntSlider(value=9, description='smoothi…

#### Generate "trench-row" detection output

After determining your desired hyperparameters, set them in the next cell and run it to produce output for later steps. **Note: The thresholding parameters do not need to be specified at this point.**

In [20]:
y_percentile = 85
smoothing_kernel_y_dim_0 = 9

y_percentiles_smoothed_list = kymo.map_to_fovs(kymo.get_smoothed_y_percentiles,imported_array_list,y_percentile,(smoothing_kernel_y_dim_0,1))

#### Tune "trench-row" cropping hyperparameters

Next, we will use the detected rows to perform cropping of the input image in the y-dimension:

1. Determine edges of trench rows based on threshold mask.
2. Filter out rows that are too small.
3. Perform cropping using the "end" of the row as reference (the end referring to the part of the trench farthest from the feeding channel).

This method uses the following `kwargs`, which you can tune here:

**triangle_nbins (int)** : Number of bins to use in the triangle method histogram.

**triangle_scaling (float)** : Scaling factor to apply to the threshold determined by the triangle method.

**y_min_edge_dist (int)** : Minimum row length necessary for detection.

**padding_y (int)** : Padding to be used when cropping in the y-dimension.

**trench_len_y (int)** : Length from the end of the tenches to be used when cropping in the y-dimension.

**top_orientation (int)** : The orientation of the top-most row where 0 corresponds to a trench with a downward-oriented trench opening and 1 corresponds to a trench with an upward-oriented trench opening.

**vertical_spacing (float)** : Parameter for setting the distance of plots being viewed.

Running the following widget will display y-cropped images for each fov and timepoint.

In [21]:
matplotlib.rcParams['figure.figsize'] = [20, 10]
interact_manual(kymo.preview_y_crop,y_percentiles_smoothed_list=fixed(y_percentiles_smoothed_list),\
                imported_array_list=fixed(imported_array_list),\
                triangle_nbins=IntSlider(value=50, min=10, max=200, step=10),\
                triangle_scaling=FloatSlider(value=1., min=0., max=2., step=0.01),\
                y_min_edge_dist=IntSlider(value=50, min=10, max=200, step=10),\
                padding_y=IntSlider(value=20, min=0, max=100, step=1),\
                trench_len_y=IntSlider(value=270, min=0, max=1000, step=10),
                top_orientation=IntSlider(value=0, min=0, max=1, step=1),
               vertical_spacing=FloatSlider(value=0.9, min=0., max=2., step=0.01))

interactive(children=(IntSlider(value=50, description='triangle_nbins', max=200, min=10, step=10), FloatSlider…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

#### Generate "trench-row" cropping output

After determining your desired hyperparameters, set them in the next cell and run it to produce output for later steps.

In [22]:
triangle_nbins=50
triangle_scaling=1.3
y_min_edge_dist=50
padding_y=40
trench_len_y=130
top_orientation=0

trench_edges_y_lists = kymo.map_to_fovs(kymo.get_trench_edges_y,y_percentiles_smoothed_list,triangle_nbins,triangle_scaling,y_min_edge_dist)
row_num_list = kymo.map_to_fovs(kymo.get_row_numbers,trench_edges_y_lists)
cropped_in_y_list = kymo.map_to_fovs(kymo.crop_y,trench_edges_y_lists,row_num_list,imported_array_list,padding_y,trench_len_y,top_orientation)

#### Tune trench detection hyperparameters

Next, we will detect the positions of trenchs in the y-cropped images as follows:

1. Reducing each 2D image to a 1D signal along the x-axis by computing the qth percentile of the data along the y-axis.
2. Determine the signal background by smooth this signal using a large median kernel.
3. Subtract the background signal.
4. Smooth the resultant signal using a median kernel.
5. Use a [otsu threhsold](https://imagej.net/Auto_Threshold#Otsu) to determine the trench midpoint poisitons.

This method uses the following `kwargs`, which you can tune here:

**x_percentile (int)** : Percentile to use for step 1.

**background_kernel_x (int)** : Median kernel size to use for step 2.

**smoothing_kernel_x (int)** : Median kernel size to use for step 4.

**otsu_nbins (int)** : Number of bins to use in the Otsu's method histogram.

**otsu_scaling (float)** : Scaling factor to apply to the threshold determined by Otsu's method.

**vertical_spacing (float)** : Parameter for setting the distance of plots being viewed.

Running the following widget will display the smoothed 1-D signal for each of your timepoints. In addition, the threshold value for each fov will be displayed as a red line.

In [23]:
interact_manual(kymo.preview_x_percentiles, cropped_in_y_list=fixed(cropped_in_y_list),t=IntSlider(value=0, min=0, max=cropped_in_y_list[0].shape[4]-1, step=1),\
                x_percentile=IntSlider(value=85, min=0, max=100, step=1),background_kernel_x=IntSlider(value=301, min=1, max=601, step=50), smoothing_kernel_x=IntSlider(value=9, min=1, max=31, step=2),\
               otsu_nbins=IntSlider(value=50, min=10, max=200, step=10),otsu_scaling=FloatSlider(value=1., min=0., max=2., step=0.01),\
               vertical_spacing=FloatSlider(value=0.9, min=0., max=2., step=0.01));

interactive(children=(IntSlider(value=0, description='t', max=7), IntSlider(value=85, description='x_percentil…

#### Generate trench detection output

After determining your desired hyperparameters, set them in the next cell and run it to produce output for later steps. **Note: The thresholding parameters do not need to be specified at this point.**

In [24]:
x_percentile = 85
background_kernel_x = 301
smoothing_kernel_x = 9

smoothed_x_percentiles_list = kymo.map_to_fovs(kymo.get_smoothed_x_percentiles,cropped_in_y_list,x_percentile,(background_kernel_x,1),(smoothing_kernel_x,1))

#### Check midpoint drift

Next, we will perform x-dimension drift correction of our detected midpoints as follows:

1. Begin at t=1
2. For $m \in \{midpoints(t)\}$ assign $n \in \{midpoints(t-1)\}$ to m if n is the closest midpoint to m at time $t-1$,
points that are not the closest midpoint to any midpoints in m will not be mapped.
3. Compute the translation of each midpoint at time.
4. Take the average of this value as the x-dimension drift from time t-1 to t.

This method uses the following `kwargs`, which you can tune here:

**otsu_nbins (int)** : Number of bins to use in the Otsu's method histogram.

**otsu_scaling (float)** : Scaling factor to apply to the threshold determined by Otsu's method.

**vertical_spacing (float)** : Parameter for setting the distance of plots being viewed.

Running the following widget will display the detected midpoints for each of your timepoints. If there is too much sparsity, or discontinuity, your drift correction will not be accurate.

In [25]:
interact_manual(kymo.preview_midpoints,smoothed_x_percentiles_list=fixed(smoothed_x_percentiles_list),\
                otsu_nbins=IntSlider(value=50, min=10, max=200, step=10),
                otsu_scaling=FloatSlider(value=1., min=0., max=2., step=0.01),\
               vertical_spacing=FloatSlider(value=0.8, min=0., max=2., step=0.01))

interactive(children=(IntSlider(value=50, description='otsu_nbins', max=200, min=10, step=10), FloatSlider(val…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

#### Generate midpoint drift output

After determining your desired hyperparameters, set them in the next cell and run it to produce output for later steps.

In [26]:
otsu_nbins = 50
otsu_scaling = 1.


all_midpoints_list = kymo.map_to_fovs(kymo.get_all_midpoints,smoothed_x_percentiles_list,otsu_nbins,otsu_scaling)
x_drift_list = kymo.map_to_fovs(kymo.get_x_drift,all_midpoints_list)

#### Tune trench cropping hyperparameters

Trench cropping simply uses the drift-corrected midpoints as a reference and crops out some fixed length around them to produce an output kymograph

This method uses the following `kwargs`, which you can tune here:

**trench_width_x (int)** : Trench width to use for cropping.

**vertical_spacing (float)** : Parameter for setting the distance of plots being viewed.

Running the following widget will display a random kymograph for each row in each fov.

In [28]:
matplotlib.rcParams['figure.figsize'] = [20, 10]
interact_manual(kymo.preview_kymographs,cropped_in_y_list=fixed(cropped_in_y_list),all_midpoints_list=fixed(all_midpoints_list),\
                x_drift_list=fixed(x_drift_list),trench_width_x=IntSlider(value=30, min=10, max=50, step=1),
               vertical_spacing=FloatSlider(value=0.8, min=0., max=2., step=0.01))

interactive(children=(IntSlider(value=30, description='trench_width_x', max=50, min=10), FloatSlider(value=0.8…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>